#### Time series classification
Implementation of Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2016, arXiv) in PyTorch by using a skorch wrapper 

In this notebook, we are using the dataset using during the IML lectures (https://github.com/IBM/sail/tree/imla_notebooks/notebooks/datasets)

Error while fitting: RuntimeError: expected scalar type Double but found Float

Authors: Marina Georgati, Hao Miao

In [1]:
import pandas as pd
import numpy as np
# import our modules
from src import data, model, utils

c:\Users\NM12LQ\Anaconda3\envs\imla\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Dataset from https://github.com/AlexImb/automl-streams/blob/master/demos/_datasets/agrawal_gen.csv
csv_file = "C:/Users/NM12LQ/OneDrive - Aalborg Universitet/PhD/PhDCourses/11. IMLA/tsc/data/agrawal_gen_abrupt.csv"

df = pd.read_csv(csv_file,
                    names=["salary", "commission", "age", "education level", "make of car", "zipcode", "house value", "years house owned", "loan", "y"])
df.head()

,salary,commission,age,education level,make of car,zipcode,house value,years house owned,loan,y
0,68757.426754,58022.846494,21.0,4.0,5.0,7.0,221792.307247,10.0,27402.870141,0
1,126298.059683,0.000000,60.0,0.0,19.0,5.0,248027.000114,6.0,197321.294347,0
2,20642.840226,65253.012270,30.0,2.0,5.0,2.0,591488.097260,17.0,36810.494158,0
3,74382.160911,67141.427442,60.0,4.0,2.0,8.0,123653.321624,4.0,354680.807306,0
4,44492.145159,83925.141993,33.0,0.0,2.0,7.0,204227.660125,11.0,360978.690701,0


In [3]:
df = pd.read_csv(csv_file,
                    names=["x0", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "y"])
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,y
0,68757.426754,58022.846494,21.0,4.0,5.0,7.0,221792.307247,10.0,27402.870141,0
1,126298.059683,0.000000,60.0,0.0,19.0,5.0,248027.000114,6.0,197321.294347,0
2,20642.840226,65253.012270,30.0,2.0,5.0,2.0,591488.097260,17.0,36810.494158,0
3,74382.160911,67141.427442,60.0,4.0,2.0,8.0,123653.321624,4.0,354680.807306,0
4,44492.145159,83925.141993,33.0,0.0,2.0,7.0,204227.660125,11.0,360978.690701,0


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = df.copy()
X.drop("y", axis=1, inplace=True)
X[X.columns] = scaler.fit_transform(X)
X

,x0,x1,x2,x3,x4,x5,x6,x7,x8
0,-0.431300,1.374318,-1.636464,1.429498,-0.785628,1.158028,-0.914743,-0.644421,-1.555556
1,1.099179,-0.737589,0.584606,-1.401639,1.643154,0.388963,-0.829445,-1.109183,-0.380831
2,-1.711064,1.637480,-1.123909,0.013929,-0.785628,-0.764636,0.287262,0.168913,-1.490517
3,-0.281692,1.706214,0.584606,1.429498,-1.306081,1.542561,-1.233826,-1.341564,0.707068
4,-1.076714,2.317105,-0.953058,-1.401639,-1.306081,1.158028,-0.971851,-0.528230,0.750608
...,...,...,...,...,...,...,...,...,...
24995,0.423851,-0.737589,0.242903,-0.693855,0.428763,-1.533701,2.318492,-1.225373,-1.398224
24996,-1.639362,0.573827,-0.668305,-0.693855,-1.306081,-1.533701,1.816275,0.866056,-0.776598
24997,0.571919,-0.737589,-1.123909,-0.693855,-1.132597,-1.149168,1.672108,1.330819,1.664225
24998,0.703634,-0.737589,0.584606,0.013929,0.775732,1.542561,-1.424730,-0.179658,-1.073691


In [5]:
# Cleaning and preprocessing
from sklearn.model_selection import train_test_split

scaler = StandardScaler()

X = df.copy()
y = X['y'].values
X.drop("y", axis=1, inplace=True)
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
y = y.reshape(len(y),1)
print(X.shape, y.shape)

(25000, 9) (25000, 1)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=0, shuffle=False)


In [8]:
time_steps = X_test.shape[-1]
n_classes = len(np.unique(y_test))
print(time_steps, n_classes)

9 2


In [9]:
import numpy as np
from time import sleep 

# Import SKORCH NN classifier
from skorch import NeuralNetClassifier
import torch
# The Neural Net is initialized with fixed hyperparameters
nn = NeuralNetClassifier(model._ConvNetModel(time_steps, n_classes), max_epochs=10, lr=0.01, batch_size=12, optimizer=torch.optim.SGD )# optim.RMSprop)
X = X_train.to_numpy().astype(np.double)
y_train= y_train.astype(np.double)
print(X.dtype)
print(y_train.dtype)
nn.fit(X, y_train)

float64
float64


RuntimeError: expected scalar type Double but found Float